In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv
/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
random_state=42

In [3]:
%pip install -q autogluon

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.0 requires botocore<1.34.23,>=1.33.2, but you have botocore 1.29.165 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-pubsub 2.19.0 requires grpcio<2.0dev,>=1.51.3, but you have grpcio 1.51.1 which is incompatible.
jupyter-event

In [4]:
%pip install --upgrade --force-reinstall -q ray==2.6.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.2 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
autogluon-multimodal 1.0.0 requires jsonschema<4.18,>=4.14, but you have jsonschema 4.21.1 which is inc

In [5]:
# %pip install -q 

In [6]:
from autogluon.tabular import TabularDataset

In [7]:
# pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Not using additional data

In [8]:
# train_path = "/kaggle/input/playground-series-s4e2/train.csv"
# test_path = "/kaggle/input/playground-series-s4e2/test.csv"

In [9]:
# df_train = TabularDataset(train_path)
# df_train = df_train.set_index(["id"])

In [10]:
# df_test = TabularDataset(test_path)
# df_test = df_test.set_index(["id"])

## Using additional data

In [11]:
train1_path = "/kaggle/input/playground-series-s4e2/train.csv"
train2_path = "/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv"

# train_path = "/kaggle/input/playground-series-s4e2/train.csv"
test_path = "/kaggle/input/playground-series-s4e2/test.csv"

In [12]:
df_train_1 = TabularDataset(train1_path)
df_train_1.drop(["id"], axis=1, inplace=True)
df_train_2 = TabularDataset(train2_path)

Loaded data from: /kaggle/input/playground-series-s4e2/train.csv | Columns = 18 / 18 | Rows = 20758 -> 20758
Loaded data from: /kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv | Columns = 17 / 17 | Rows = 2111 -> 2111


In [13]:
df_train = pd.concat([df_train_1, df_train_2])

In [14]:
df_train.shape

(22869, 17)

In [15]:
df_train.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [16]:
df_test = TabularDataset(test_path)
df_test = df_test.set_index(["id"])

Loaded data from: /kaggle/input/playground-series-s4e2/test.csv | Columns = 17 / 17 | Rows = 13840 -> 13840


In [17]:
df_test.shape

(13840, 16)

In [18]:
df_test.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
id,,,,,,,,,,,,,,,,
20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


# Feature Engineering

### CALC special value

In [19]:
df_train.CALC.value_counts()

CALC
Sometimes     16467
no             5802
Frequently      599
Always            1
Name: count, dtype: int64

In [20]:
df_test.CALC.value_counts()

CALC
Sometimes     9979
no            3513
Frequently     346
Always           2
Name: count, dtype: int64

In [this comment](https://www.kaggle.com/competitions/playground-series-s4e2/discussion/472597#2631021), the author suggest replace "Always" with "Frequently". 

In [21]:
df_train["CALC"] = np.where(df_train["CALC"] == "Always", "Frequently", df_train["CALC"])
df_test["CALC"] = np.where(df_test["CALC"] == "Always", "Frequently", df_test["CALC"])

# Pre-processing

In [22]:
df_train.CALC.value_counts()

CALC
Sometimes     16467
no             5802
Frequently      600
Name: count, dtype: int64

In [23]:
df_test.CALC.value_counts()

CALC
Sometimes     9979
no            3513
Frequently     348
Name: count, dtype: int64

## Additional features

Building special features based on [this comment](https://www.kaggle.com/competitions/playground-series-s4e2/discussion/473220)

### BMI - Body Mass Index

In [24]:
df_train["BMI"] = df_train["Weight"] / (df_train["Height"] * df_train["Height"])
df_test["BMI"] = df_test["Weight"] / (df_test["Height"] * df_test["Height"])

### PAL - Physical Activity Level

In [25]:
df_train["TUE"].describe(), df_train["FAF"].describe()

(count    22869.000000
 mean         0.620551
 std          0.602850
 min          0.000000
 25%          0.000000
 50%          0.579541
 75%          1.000000
 max          2.000000
 Name: TUE, dtype: float64,
 count    22869.000000
 mean         0.984382
 std          0.839466
 min          0.000000
 25%          0.015860
 50%          1.000000
 75%          1.596576
 max          3.000000
 Name: FAF, dtype: float64)

In [26]:
df_test["TUE"].describe(), df_test["FAF"].describe()

(count    13840.000000
 mean         0.611033
 std          0.608005
 min          0.000000
 25%          0.000000
 50%          0.552498
 75%          1.000000
 max          2.000000
 Name: TUE, dtype: float64,
 count    13840.000000
 mean         0.974532
 std          0.840361
 min          0.000000
 25%          0.001086
 50%          1.000000
 75%          1.571865
 max          3.000000
 Name: FAF, dtype: float64)

In [27]:
df_train["PAL"] = df_train["TUE"] - df_train["FAF"]
df_test["PAL"] = df_test["TUE"] - df_test["FAF"]

### MH - Meal Habits

In [28]:
df_train["FCVC"].describe(), df_train["NCP"].describe()

(count    22869.000000
 mean         2.443428
 std          0.533329
 min          1.000000
 25%          2.000000
 50%          2.392665
 75%          3.000000
 max          3.000000
 Name: FCVC, dtype: float64,
 count    22869.000000
 mean         2.754344
 std          0.712711
 min          1.000000
 25%          3.000000
 50%          3.000000
 75%          3.000000
 max          4.000000
 Name: NCP, dtype: float64)

In [29]:
df_test["FCVC"].describe(), df_test["NCP"].describe()

(count    13840.000000
 mean         2.442898
 std          0.531606
 min          1.000000
 25%          2.000000
 50%          2.358087
 75%          3.000000
 max          3.000000
 Name: FCVC, dtype: float64,
 count    13840.000000
 mean         2.750610
 std          0.710927
 min          1.000000
 25%          3.000000
 50%          3.000000
 75%          3.000000
 max          4.000000
 Name: NCP, dtype: float64)

In [30]:
df_train["MH"] = df_train["FCVC"] + df_train["NCP"]
df_train["MH"] = df_test["FCVC"] + df_test["NCP"]

### HNH - Healthy Nutrition Habits

In [31]:
df_train["FCVC"].describe(), df_train["FAVC"].describe()

(count    22869.000000
 mean         2.443428
 std          0.533329
 min          1.000000
 25%          2.000000
 50%          2.392665
 75%          3.000000
 max          3.000000
 Name: FCVC, dtype: float64,
 count     22869
 unique        2
 top         yes
 freq      20848
 Name: FAVC, dtype: object)

In [32]:
df_test["FCVC"].describe(), df_test["FAVC"].describe()

(count    13840.000000
 mean         2.442898
 std          0.531606
 min          1.000000
 25%          2.000000
 50%          2.358087
 75%          3.000000
 max          3.000000
 Name: FCVC, dtype: float64,
 count     13840
 unique        2
 top         yes
 freq      12583
 Name: FAVC, dtype: object)

In [33]:
df_train["FAVC"].unique()

array(['yes', 'no'], dtype=object)

In [34]:
df_train["FAVC"] = np.where(df_train["FAVC"] == "yes", 1, 0)
df_test["FAVC"] = np.where(df_test["FAVC"] == "yes", 1, 0)

In [35]:
df_train["FAVC"].unique()

array([1, 0])

In [36]:
df_train["HNH"] = df_train["FCVC"] + df_train["FAVC"]
df_test["HNH"] = df_test["FCVC"] + df_test["FAVC"]

### TUS - Tech Usage Score

In [37]:
df_train["TUE"].describe(), df_test["TUE"].describe()

(count    22869.000000
 mean         0.620551
 std          0.602850
 min          0.000000
 25%          0.000000
 50%          0.579541
 75%          1.000000
 max          2.000000
 Name: TUE, dtype: float64,
 count    13840.000000
 mean         0.611033
 std          0.608005
 min          0.000000
 25%          0.000000
 50%          0.552498
 75%          1.000000
 max          2.000000
 Name: TUE, dtype: float64)

In [38]:
df_train["TUS"] = df_train["Age"] * df_train["TUE"]
df_test["TUS"] = df_test["Age"] * df_test["TUE"]

## Dropping unused features

In [39]:
# features_to_drop = [
#     "Weight",
#     "Height",
# #     "CH2O"
#     "SMOKE"
# ]

# df_train.drop(features_to_drop, axis=1, inplace=True)
# df_test.drop(features_to_drop, axis=1, inplace=True)

# Modeling

In [40]:
from autogluon.tabular import TabularPredictor

In [41]:
label = "NObeyesdad"
print(f"Unique classes: {list(df_train[label].unique())}")

Unique classes: ['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight', 'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_I']


## Building predictor

In [42]:
predictor = TabularPredictor(
    label=label,
    eval_metric="accuracy"
).fit(
    train_data=df_train,
    num_bag_folds=5,
    num_bag_sets=5,
    auto_stack=True,
    refit_full=True,
    set_best_to_refit_full=True,
    num_gpus=2,
    ag_args_fit={'num_gpus': 2},
    presets="best_quality"
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240207_102435"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=5
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240207_102435/ds_sub_fit/sub_fit_ho.
2024-02-07 10:24:36,491	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

In [43]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20240207_020617")

In [44]:
predictor.leaderboard(df_train)

INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForest_r195_BAG_L1_FULL,0.999913,NaN,accuracy,0.445739,0.901085,16.420688,0.445739,0.901085,16.420688,1,True,54
1,RandomForestEntr_BAG_L1_FULL,0.999913,NaN,accuracy,0.500871,0.897229,6.596564,0.500871,0.897229,6.596564,1,True,35
2,RandomForest_r195_BAG_L1,0.999913,0.902313,accuracy,0.515917,0.901085,16.420688,0.515917,0.901085,16.420688,1,True,26
3,RandomForestEntr_BAG_L1,0.999913,0.908304,accuracy,0.523952,0.897229,6.596564,0.523952,0.897229,6.596564,1,True,7
4,ExtraTrees_r42_BAG_L1_FULL,0.999913,NaN,accuracy,0.525586,0.997068,4.770268,0.525586,0.997068,4.770268,1,True,50
5,RandomForestGini_BAG_L1,0.999913,0.907823,accuracy,0.528397,0.916451,5.209540,0.528397,0.916451,5.209540,1,True,6
6,ExtraTrees_r42_BAG_L1,0.999913,0.905112,accuracy,0.649642,0.997068,4.770268,0.649642,0.997068,4.770268,1,True,22
7,RandomForestGini_BAG_L1_FULL,0.999913,NaN,accuracy,0.657039,0.916451,5.209540,0.657039,0.916451,5.209540,1,True,34
8,ExtraTreesGini_BAG_L1,0.999913,0.901745,accuracy,0.808178,1.028682,2.958858,0.808178,1.028682,2.958858,1,True,9
9,ExtraTreesEntr_BAG_L1_FULL,0.999913,NaN,accuracy,0.808479,1.028311,2.782129,0.808479,1.028311,2.782129,1,True,38


In [45]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                               model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                WeightedEnsemble_L2   0.916568    accuracy      32.935905  809.159832                0.003834          10.230963            2       True         28
1                     XGBoost_BAG_L1   0.914294    accuracy       0.166245   39.226504                0.166245          39.226504            1       True         11
2               LightGBM_r131_BAG_L1   0.913945    accuracy       5.994753  173.989445                5.994753         173.989445            1       True         16
3                 XGBoost_r33_BAG_L1   0.913289    accuracy       0.338573  106.336636                0.338573         106.336636            1       True         21
4                    LightGBM_BAG_L1   0.913158    accuracy       1.787043   83.128545                1.787043   

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [46]:
# df_train.duplicate()

In [47]:
# predictor.feature_importance(df_train)

Barchart showing feature importance

In [48]:
# feature_importance = predictor.feature_importance(df_train)

In [49]:
# feature_importance["importance"]

In [50]:
# sns.barplot(feature_importance, x = "importance", y=feature_importance.index)

## Predict the test set

In [51]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20240206_104954")

In [52]:
predictions = predictor.predict(df_test)

INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [53]:
predictions

id
20758        Obesity_Type_II
20759     Overweight_Level_I
20760       Obesity_Type_III
20761         Obesity_Type_I
20762       Obesity_Type_III
                ...         
34593    Overweight_Level_II
34594          Normal_Weight
34595    Insufficient_Weight
34596          Normal_Weight
34597        Obesity_Type_II
Name: NObeyesdad, Length: 13840, dtype: object

In [54]:
preds = []
for pred in predictions:
    preds.append(str(pred))

In [55]:
# preds

In [56]:
predict_df = pd.DataFrame({
    "id": df_test.index.values,
    "NObeyesdad": preds
})

In [57]:
# with open("submission.csv", "w+") as f:
#     print(f"id,NObeyesdad", file=f)
#     for idx, pred in zip(df_test.index.values, preds):
#         print(f"{idx},{pred}", file=f)

In [58]:
predict_df.to_csv("submission.csv", index=False)